# PACBED Simulation

In [1]:
# Import required packages
import numpy as np
from tqdm.notebook import tqdm
import os
import shutil
from PIL import Image
import glob
import subprocess
import pandas as pd
from ase.io import read
from ase.build import surface
from abtem import *
import cupy as cp
import matplotlib.pyplot as plt

In [2]:
# Input Parameters - Generating Trainings Dataset
parameters_simulation = {
    'voltage_HT' : 300,                               # [kV]
    'conv_angle' : [15.0, 25.0],                         # [mrad] minimum and maximum convergence angle
    'conv_step' : 0.5,                               # [mrad] convergence angle stepsize
    'thickness' : 100,                              # [nm]   maximum thickness
    'thickness_step' : 1,                            # [nm]   thickness stepsize
    'mistilt' : [0, 10],                             # [mrad] minimum and maximum mistilt
    'mistilt_step' : 1,                              # [mrad] mistilt stepsize
    'azimuth' : [0, 0.5],                           # [] Direction of the mistilt (1.. full circle)
    'azimuth_step' : 0.1,                           # [] azimuth stepsize
    'material' : 'Strontium titanate',                           # Material
    'composition' : 'SrTiO3',
    'direction' : (0, 0, 1),                         # Direction --> if cell is not square output is strecthed 
    'crystal_file' : '.\\crystal_files\\SrTiO3_mp-5229_computed.cif', # path of the crystal structure
    'dim' : (170, 170),                              # dimension of the image for saving 
    }

In [49]:
class PACBED_simulator:
    def __init__(self, parameters):
        # Declare variables
        self.Voltage = parameters['voltage_HT']
        self.Conv_angle = parameters['conv_angle']
        self.Conv_step = parameters['conv_step']
        self.Thickness = parameters['thickness'] 
        self.Thickness_step = parameters['thickness_step']
        self.Mistilt = parameters['mistilt']
        self.Mistilt_step = parameters['mistilt_step']
        self.Azimuth = parameters['azimuth']
        self.Azimuth_step = parameters['azimuth_step']
        self.Material = parameters['material']
        self.Composition = parameters['composition']
        self.Direction = parameters['direction']
        self.Dim = parameters['dim']
        
        self.Crystal_path = parameters['crystal_file']
        
        self.df = None
        self.column_df = None
        self.atoms_aligned = None
        self.nx = None
        self.ny = None
        self.id_reg = None
        
        # Estimate final required memory
        n = (((self.Conv_angle[1] - self.Conv_angle[0])/self.Conv_step + 1) * 
            ((self.Thickness)/self.Thickness_step + 1) * 
            ((self.Mistilt[1] - self.Mistilt[0])/self.Mistilt_step + 1) * 
            ((self.Azimuth[1] - self.Azimuth[0])/self.Azimuth_step + 1))
        memory_factor = 3.06368e-5/(170*170) * self.Dim[0] * self.Dim[1]  # Estimated from previous calculations    
        print("{:.0f} images will be simulated.".format(n))
        print("Estimated required memory: {:.2f} GB".format(n*memory_factor))
        
        # Create dataframe
        self.column_df = ['Path', 'Thickness', 'Mistilt', 'Conv_Angle', 'Electron_Energy', 'Azimuth', 'Direction', 'Material']
        self.df = pd.DataFrame(columns=self.column_df)
        
    def make_register(self):
        # Load csv file, containing all systems/models
        df_indexer = pd.read_csv(os.path.join('.\\data','Register.csv'), sep = ';', index_col = 'id')

        # Make entry
        
        # Generate id
        if df_indexer.empty:
            self.id_reg = 0
        else:
            self.id_reg = np.amax(df_indexer.index) + 1

        # Saving directory
        self.Saving_path = os.path.join('.\\data', str(self.id_reg))

        data_register = [self.Saving_path, self.Material, self.Composition, self.Direction, self.Thickness, self.Thickness_step, self.Voltage, self.Conv_angle[0], self.Conv_angle[1], self.Conv_step, self.Mistilt[0], self.Mistilt[1], self.Mistilt_step, self.Azimuth[0], self.Azimuth[1], self.Azimuth_step, self.Dim]
        df_system = pd.DataFrame(data = [data_register], columns = df_indexer.columns, index = [self.id_reg])

        # Add entry
        # Check if requested PACBEDs are already available and if boundaries make sense
        df_indexer = pd.concat([df_indexer, df_system], ignore_index=False)
        df_indexer.to_csv('.\\data\\Register.csv', sep = ';', index = True, index_label = 'id')      
        
        # Create folder for savings
        os.mkdir(self.Saving_path)
        self.path_sim = os.path.join(self.Saving_path, 'simulation')
        os.mkdir(self.path_sim)
        
        # Copy crystal file
        shutil.copy2(self.Crystal_path, os.path.join(self.Saving_path, 'crystal_file.cif'))
        
        # Save register entry
        df_system.to_csv(os.path.join(self.Saving_path, f'Register_{self.id_reg}.csv'), sep = ';', index = True, index_label = 'id')
        display(df_system)
       
    def construct_supercell(self, nx = 16, ny = 16, visualize = True):
        self.nx = nx
        self.ny = ny
        
        # Load crystal structure
        atoms = read(os.path.join(self.Saving_path, 'crystal_file.cif'))
        
        # Align to given direction (propagation through z-direction)
        self.atoms_aligned = surface(atoms, indices=self.Direction, layers=1, periodic=True)
        
        # Adapting replication nx and ny to get a squared supercell
        if self.atoms_aligned.cell[0][0] > self.atoms_aligned.cell[1][1]:
            self.ny = np.round(self.ny * self.atoms_aligned.cell[0][0]/self.atoms_aligned.cell[1][1]).astype(int)
        elif self.atoms_aligned.cell[0][0] < self.atoms_aligned.cell[1][1]:
            self.nx = np.round(self.nx * self.atoms_aligned.cell[1][1]/self.atoms_aligned.cell[0][0]).astype(int)

        
        # Calculate number of layers for given thickness
        thickness_unit_cell = self.atoms_aligned.cell[-1][-1] / 10 # conversion to nm

        # Required layers of unit cell
        n_layers = np.round(self.Thickness/thickness_unit_cell).astype(int)

        # Replicate cell to required thickness and lateral for better PACBED resolution
        self.atoms_aligned *= (self.nx, self.ny, n_layers)
        
        if visualize:
            atoms_plot = self.atoms_aligned
            # Center atoms in cell and add vacuum margin
            atoms_plot.center(axis=2, vacuum=3)
            
            # Create figure
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,10))
            show_atoms(atoms_plot, plane='xy', ax = ax1)
            ax1.set_title('Top view: xy')
            show_atoms(atoms_plot, plane='yz', ax = ax2)
            ax2.set_title('Side view: yz')
            
        print('Supercell generated with [{}, {}, {}] unit cells oriented in {}.'.format(self.nx, self.ny, n_layers, self.Direction))
    
    def simulate_PACBED(self):
        # Create Potential
        potential = Potential(self.atoms_aligned, 
                              gpts=512,
                              device='gpu', 
                              projection='infinite', 
                              slice_thickness= 0.5, # Default 0.5 A
                              parametrization='kirkland', 
                              storage  = 'cpu')
        
        # Save after every defined thickness by n_steps slice
        n_step = np.round(self.atoms_aligned.cell[-1][-1]/(self.Thickness_step * 10)/(potential.slice_thickness*10)).astype(int)
        chunks = [(i, i + n_step) for i in range(0, len(potential),n_step)] 
        
        # Thickness in Angstrom at each chunk
        z_vec = np.round(np.array(chunks)[:,0] * self.atoms_aligned.cell[-1][-1] / np.max(np.array(chunks)[:,0])).astype(int)
        
        # Slice potential (PACBED pattern at each slice)
        potential_slices = []
    
        for chunk_idx, (start_slice, end_slice) in enumerate(tqdm(chunks, desc = 'Slicing the potential')):
            potential_slices.append(potential[start_slice:end_slice])
        
        
        # Define detector
        detector = PixelatedDetector(max_angle=50)

        # Create scanning grid
        end = (potential.extent[0] / self.nx, potential.extent[1] / self.ny)
        scan = GridScan(start=[0, 0], end=end, sampling=0.6) # Sampling influence number of CBEDs for PACBED
        
        # Create vectors:
        conv_angle_vec = np.arange(self.Conv_angle[0],self.Conv_angle[1] + self.Conv_step, self.Conv_step)
        thickness_vec = np.arange(0, self.Thickness + self.Thickness_step, self.Thickness_step)
        mistilt_vec = np.arange(self.Mistilt[0],self.Mistilt[1] + self.Mistilt_step, self.Mistilt_step)
        azimuth_vec = np.arange(self.Azimuth[0],self.Azimuth[1] + self.Azimuth_step, self.Azimuth_step)               

        print ('Generating PACBEDs:')
        # Loop over azimuth angle
        for idx_azi, azimuth in enumerate(tqdm(azimuth_vec, desc = 'Progress Azimuth', position=0)):
            # Loop over mistilt angle
            for idx_mis, mistilt in enumerate(tqdm(mistilt_vec, desc = 'Progress Mistilt', position=1, leave = False)):
                # Calculate mistilt in x and y direction
                azimuth_angle = 2*np.pi*azimuth # Azimuth in radians
                mistilt_x = mistilt * np.sin(azimuth_angle)
                mistilt_y = mistilt * np.cos(azimuth_angle)
                
                # Loop over convergence angle
                for idx_conv, conv in enumerate(tqdm(conv_angle_vec, desc = 'Progress Convergence Angle', position=2, leave = False)):  
                    
                    # Generate new probe
                    probe = Probe(energy=self.Voltage * 1000, semiangle_cutoff= conv, tilt = (mistilt_x, mistilt_y), device='gpu')
                    probe.grid.match(potential)
        

                    # PACBEDS are save in measurements
                    measurements = []
                    
                    # Build probe wave functions at the provided positions (occur memory errors split positions)
                    waves = probe.build(scan.get_positions())

                    for idx_chunk in range(0,len(potential_slices)):
                        # Start multislice simulation
                        waves = waves.multislice(potential_slices[idx_chunk], pbar=False)
                        # Calculate the far field intensity of the wave functions
                        measurement_chunk = detector.detect(waves)
                        
                        # Sum over all positions (CBED to PACBED)
                        measurements.append(measurement_chunk.sum((0)))
                        
                    
                    # Save tiff
                    self.tiff_saving(measurements, conv, mistilt, azimuth_angle, z_vec)
    
 
        # Path for dataframe
        path_df = os.path.join(self.path_sim, 'df.csv')
        # Save dataframe
        self.df.to_csv(path_df, sep = ';', index=False)
        print('Dataframe saved at {}'.format(path_df))
    
    def tiff_saving(self, measurements, conv, mistilt, azimuth_angle, z_vec):
        for i in range(0, len(measurements)):
            # Convert cupy to numpy array (GPU to CPU)
            measurements_arr = measurements[i].get()
            
            # Normalize
            measurements_arr = (255*measurements_arr/np.amax(measurements_arr)).astype(np.uint8)            
            
            # Convert to PIL-framework
            measurments_img = Image.fromarray(measurements_arr)
            
            # Resize
            measurments_img = measurments_img.resize(self.Dim, resample = Image.BICUBIC)

            # Create folder to save PACBEDs as tiff
            subfolder_name = "Tilt_{}_Azimuth_{:.0f}_Conv_Angle_{}".format(mistilt, 1000 * azimuth_angle, conv)            
            path_tiff = os.path.join(self.path_sim, subfolder_name)   
            if not os.path.exists(path_tiff):
                os.mkdir(path_tiff)


            # Create path + name for every file
            path_tiff_file = os.path.join(path_tiff, str(z_vec[i]).zfill(4) + '_Angstrom.tif')

            
            # Save tiff
            measurments_img.save(path_tiff_file)

            # Add entry in dataframe
            df_row = pd.DataFrame([[path_tiff_file, z_vec[i], mistilt, conv, self.Voltage, 1000 * azimuth_angle, self.Direction, self.Material]], columns = self.column_df)
            self.df = pd.concat([self.df, df_row], ignore_index=True)


## Simulating PACBEDs

In [50]:
# Initialise parameters and estimate required memory
simulator = PACBED_simulator(parameters_simulation)

139986 images will be simulated.
Estimated required memory: 4.29 GB


In [51]:
# Generates folders for saving results and entry into the register
simulator.make_register()

,path,material,composition,direction,thickness,thickness step,high tension,convergenc angle min,convergenc angle max,convergenc angle step,mistilt min,mistilt max,mistilt step,azimuth min,azimuth max,azimuth step,dim
2,.\data\2,Strontium titanate,SrTiO3,"(0, 0, 1)",100,1,300,15.0,25.0,0.5,0,10,1,0,0.5,0.1,"(170, 170)"


In [ ]:
# Generate supercell
simulator.construct_supercell(nx = 16, ny = 16, visualize = False) # Visualization can take long, depending on the number of atoms

In [ ]:
# Simulates PACBEDs and saves them to given path
simulator.simulate_PACBED()